In [1]:
import pandas as pd
import numpy as np

import copy

from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_parquet('../../data/final_data/fbref/player_match_reports.parquet')
df['Date'] = pd.to_datetime(df['Date'], format = "%Y-%m-%d")
df

,Season,League,Team,Date,Player,#,Nation,Pos,Age,Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Aerial_Duels_Won%,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2020-2021,EPL,Fulham,2020-09-12,Aboubakar Kamara,47,mr MTN,FW,25-189,62.0,0,0,0,0,1,0,0,0,26.0,13.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,16.0,87.5,0.0,20.0,2.0,0.0,2.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,13.0,0.0,0.0,1.0,5.0,7.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,3,1,0,0,0,0.0,0.0,0,4.0,2.0,1.0,66.7,14.0,16.0,87.5,189.0,12.0,4.0,5.0,80.0,7.0,8.0,87.5,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,12.0,4.0,0.0,0.0,5.0,0.0,0,0.0,0.0,0.0,0.0,11.0,4.0,1.0,4.0,10.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,1.0,26.0,0.0,1.0,13.0,14.0,1.0,22.0,0.0,0.0,0.0,45.0,23.0,0.0,0.0,3.0,4.0,26.0,18.0,69.2,2.0
1,2020-2021,EPL,Fulham,2020-09-12,Aleksandar Mitrović,9,rs SRB,FW,25-362,28.0,0,0,0,0,1,1,0,0,7.0,6.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,60.0,0.0,6.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,6.0,0.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,2,0,1,0,0.0,0.0,0,0.0,0.0,0.0,NaN,3.0,5.0,60.0,44.0,2.0,2.0,3.0,66.7,1.0,1.0,100.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,2.0,1.0,1,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,3.0,0.0,0.0,1.0,3.0,0.0,1.0,1.0,0.0,7.0,0.0,0.0,3.0,4.0,0.0,7.0,100.0,1.0,0.0,16.0,3.0,0.0,0.0,1.0,0.0,12.0,7.0,58.3,1.0
2,2020-2021,EPL,Fulham,2020-09-12,Ivan Cavaleiro,17,pt POR,LW,26-330,90.0,0,0,0,0,0,0,0,0,41.0,14.0,0.0,0,1.0,0.0,0.0,0.1,3.0,0.0,28.0,35.0,80.0,2.0,30.0,4.0,3.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,14.0,1.0,7.1,2.0,5.0,7.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0.0,0,1,4,0,2,0,0.0,0.0,0,6.0,1.0,0.0,100.0,28.0,35.0,80.0,485.0,84.0,14.0,15.0,93.3,11.0,13.0,84.6,3.0,6.0,50.0,0,0.1,2.0,1.0,1.0,1.0,2.0,35.0,34.0,1.0,1.0,0.0,7.0,2.0,2,0.0,0.0,0.0,0.0,27.0,2.0,6.0,4.0,29.0,2.0,0.0,0.0,28.0,0.0,0.0,0.0,1.0,41.0,4.0,11.0,23.0,11.0,0.0,41.0,75.0,3.0,0.0,189.0,76.0,2.0,0.0,2.0,1.0,41.0,34.0,82.9,1.0
3,2020-2021,EPL,Fulham,2020-09-12,Neeskens Kebano,7,cd COD,RW,28-186,62.0,0,0,0,0,1,0,0,0,30.0,9.0,1.0,0,2.0,0.1,0.1,0.0,1.0,0.0,19.0,23.0,82.6,1.0,21.0,3.0,2.0,3.0,1.0,1,1.0,0.0,0.0,1.0,2.0,50.0,1.0,9.0,3.0,33.3,1.0,6.0,2.0,2.0,0.0,0.0,2.0,0,1.0,1.0,0.0,0,0,2,1,1,1,0.0,0.0,0,4.0,0.0,0.0,NaN,19.0,23.0,82.6,215.0,39.0,15.0,15.0,100.0,3.0,6.0,50.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,23.0,22.0,1.0,1.0,0.0,6.0,0.0,1,0.0,0.0,0.0,0.0,17.0,4.0,2.0,8.0,15.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,30.0

In [4]:
map_dict = {'DM': 'M',
 'CB': 'D',
 'LB': 'D',
 'RM': 'M',
 'DF': 'D',
 'RB': 'D',
 'AM': 'M',
 'MF': 'M',
 'CM': 'M',
 'LW': 'A',
 'FW': 'A',
 'RW': 'A',
 'WB': 'M',
 'GK': 'G',
 'LM': 'M'}
new_pos = []
for i, row in df[['Pos']].iterrows():
    pos = row['Pos']
    counts = {
        'D': 0,
        'M': 0,
        'A': 0, 
        'G': 0
    }
    for p in pos.split(','):
        counts[map_dict[p]] += 1
    max_val = 0
    best_key = 'M'
    for key in counts:
        if counts[key] > max_val:
            max_val = counts[key]
            best_key = key
    new_pos.append(best_key)
    
#     print(pos, best_key)
df['PosNew'] = new_pos

In [5]:
columns_to_be_dropped = [
    'Nation',
    '#',
    'Performance_Gls',
    'Performance_Ast',
    'Performance_PK', 
    'Performance_PKatt',
] + [col for col in df.columns if '%' in col ]

df = df.drop(columns_to_be_dropped, 1)

In [6]:
cols_to_be_normalized = list(set(df.columns) - set(['Season',
 'League',
 'Team',
 'Date',
 'Player',
 'Pos',
 'PosNew',
 'Age']))
df[cols_to_be_normalized] = df[cols_to_be_normalized].div(df['Min'], axis=0)

In [7]:
df[(df['Season'] == '2017-2018') & (df.isna().any(axis=1))]

,Season,League,Team,Date,Player,Pos,Age,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,PosNew
32057,2017-2018,EPL,Tottenham,2017-10-14,Georges-Kévin N'Koudou,AM,22-243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
34714,2017-2018,EPL,West Brom,2017-12-13,Sam Field,MF,19-219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
40429,2017-2018,EPL,Manchester City,2018-05-13,Ederson,GK,24-269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G
69680,2017-2018,Ligue1,Monaco,2017-08-18,Youri Tielemans,FW,20-103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
69763,2017-2018,Ligue1,Nantes,2017-08-19,Mariusz Stępiński,FW,22-099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [8]:
df = df.dropna()

In [9]:
matches_df = pd.read_csv('../../data/final_data/fbref/matches.csv', parse_dates=['Date'])
matches_df = matches_df.rename(columns={
    'HomeScore': 'Goal_Home', 
    'AwayScore': 'Goal_Away', 
    'HomeTeam': 'Team_Home',
    'AwayTeam': 'Team_Away',
})
matches_df['Win_Home'] = 1 * ((matches_df['Goal_Home'] - matches_df['Goal_Away']) > 0)
matches_df['Win_Away'] = 1 * ((- matches_df['Goal_Home'] + matches_df['Goal_Away']) > 0)
matches_df['game_id'] = matches_df.index
matches_df.head()

,League,Season,Wk,Day,Date,Time,Team_Home,xG_Home,xG_Away,Team_Away,Attendance,Venue,Referee,Match Report,Goal_Home,Goal_Away,Round,Win_Home,Win_Away,game_id
0,EPL,2020-2021,1.0,Sat,2020-09-12,12:30,Fulham,0.2,1.8,Arsenal,NaN,Craven Cottage,Chris Kavanagh,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,3.0,NaN,0,1,0
1,EPL,2020-2021,1.0,Sat,2020-09-12,15:00,Crystal Palace,0.7,0.8,Southampton,NaN,Selhurst Park,Jonathan Moss,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.0,NaN,1,0,1
2,EPL,2020-2021,1.0,Sat,2020-09-12,17:30,Liverpool,3.3,0.6,Leeds United,NaN,Anfield,Michael Oliver,https://fbref.com/en/matches/21b58926/Liverpoo...,4.0,3.0,NaN,1,0,2
3,EPL,2020-2021,1.0,Sat,2020-09-12,20:00,West Ham,1.1,1.5,Newcastle Utd,NaN,London Stadium,Stuart Attwell,https://fbref.com/en/matches/78495ced/West-Ham...,0.0,2.0,NaN,0,1,3
4,EPL,2020-2021,1.0,Sun,2020-09-13,14:00,West Brom,0.5,2.2,Leicester City,NaN,The Hawthorns,Anthony Taylor,https://fbref.com/en/matches/7dd01ca9/West-Bro...,0.0,3.0,NaN,0,1,4


In [10]:
len(matches_df)

7314

In [11]:
matches_df_long = pd.wide_to_long(matches_df, ["Goal", "xG", 'Team', 'Win'], i="game_id", j="isHome", sep='_', suffix='(!?Home|Away)')
matches_df_long = matches_df_long.drop(['Attendance', 'Round'], 1)
matches_df_long = matches_df_long.reset_index()
matches_df_long.head()


,game_id,isHome,Date,Match Report,Day,Time,Wk,Referee,Venue,Season,League,Goal,xG,Team,Win
0,0,Home,2020-09-12,https://fbref.com/en/matches/bf52349b/Fulham-A...,Sat,12:30,1.0,Chris Kavanagh,Craven Cottage,2020-2021,EPL,0.0,0.2,Fulham,0
1,1,Home,2020-09-12,https://fbref.com/en/matches/db261cb0/Crystal-...,Sat,15:00,1.0,Jonathan Moss,Selhurst Park,2020-2021,EPL,1.0,0.7,Crystal Palace,1
2,2,Home,2020-09-12,https://fbref.com/en/matches/21b58926/Liverpoo...,Sat,17:30,1.0,Michael Oliver,Anfield,2020-2021,EPL,4.0,3.3,Liverpool,1
3,3,Home,2020-09-12,https://fbref.com/en/matches/78495ced/West-Ham...,Sat,20:00,1.0,Stuart Attwell,London Stadium,2020-2021,EPL,0.0,1.1,West Ham,0
4,4,Home,2020-09-13,https://fbref.com/en/matches/7dd01ca9/West-Bro...,Sun,14:00,1.0,Anthony Taylor,The Hawthorns,2020-2021,EPL,0.0,0.5,West Brom,0


In [12]:
len(matches_df_long)

14628

In [13]:
df_grouped = df.groupby(['League', 'Season', 'Team', 'Date', 'PosNew']).sum().reset_index()
team_stats_df = pd.merge(df_grouped, matches_df_long[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']], how='left', on=['League', 'Season', 'Team', 'Date'])
team_stats_df.head(5)


,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
0,Bundesliga,2017-2018,Augsburg,2017-08-19,A,4.0,0.057431,0.011111,0.000000,0.0,2.399867,0.728161,0.011111,0.011111,0.024098,0.003521,0.003521,0.008803,0.182551,0.000000,1.021134,1.776313,0.092641,1.810523,0.277944,0.124098,0.269564,0.011111,0.011111,0.011111,0.000000,0.000000,0.000000,0.033333,0.033333,0.728161,0.137085,0.147342,0.364080,0.216739,0.024098,0.000000,0.0,0.024098,0.011111,0.022222,0.035209,0.0,0.0,0.066667,0.055556,0.000000,0.376235,0.011111,0.0,0.0,0.0,0.464247,0.148196,0.137085,1.021134,1.776313,15.497336,3.913198,0.591242,0.757909,0.361350,0.664413,0.046320,0.261350,0.0,0.008803,0.121368,0.061183,0.143590,0.099145,0.092641,1.776313,1.591886,0.184427,0.102897,0.0,0.253824,0.011111,0.376235,0.025974,0.012987,0.012987,0.0,0.869186,0.562371,0.344755,0.522555,1.061117,0.048196,0.033333,0.022222,1.021134,0.000000,0.011111,0.000000,0.081530,2.399867,0.000000,0.138961,0.822844,1.673438,0.265956,2.215440,0.135209,0.024098,8.346831,4.807604,0.099145,0.022222,0.200167,0.046320,2.337496,1.623343,0.709713,3970,0.0,0,Away
1,Bundesliga,2017-2018,Augsburg,2017-08-19,D,4.0,0.022222,0.000000,0.022222,0.0,3.266667,0.455556,0.088889,0.033333,0.066667,0.000000,0.000000,0.003333,0.111111,0.000000,2.111111,2.755556,0.322222,1.833333,0.255556,0.033333,0.033333,0.088889,0.066667,0.066667,0.011111,0.011111,0.033333,0.066667,0.033333,0.455556,0.200000,0.233333,0.144444,0.077778,0.066667,0.022222,0.0,0.044444,0.033333,0.122222,0.233333,0.0,0.0,0.088889,0.033333,0.011111,0.133333,0.066667,0.0,0.0,0.0,0.444444,0.088889,0.122222,2.111111,2.755556,52.500000,20.277778,0.555556,0.688889,0.900000,1.022222,0.644444,0.966667,0.0,0.003333,0.077778,0.255556,0.100000,0.055556,0.322222,2.755556,2.344444,0.411111,0.077778,0.0,0.222222,0.177778,0.133333,0.055556,0.044444,0.000000,0.0,1.355556,0.655556,0.744444,1.088889,1.266667,0.100000,0.277778,0.000000,2.111111,0.000000,0.055556,0.044444,0.100000,3.266667,0.077778,1.211111,1.577778,0.755556,0.088889,2.866667,0.033333,0.000000,10.866667,6.311111,0.088889,0.011111,0.033333,0.055556,2.100000,1.966667,0.111111,3970,0.0,0,Away
2,Bundesliga,2017-2018,Augsburg,2017-08-19,G,1.0,0.011111,0.000000,0.000000,0.0,0.488889,0.000000,0.000000,0.000000,0.000000,0.001111,0.001111,0.000000,0.000000,0.000000,0.300000,0.466667,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [14]:
len(team_stats_df.groupby(['League', 'Season', 'Team', 'Date']).mean())

13958

In [15]:
home_stats_df = team_stats_df[team_stats_df['isHome'] == 'Home']
home_stats_df = home_stats_df.sort_values(['game_id', 'PosNew'])
away_stats_df = team_stats_df[team_stats_df['isHome'] == 'Away']
away_stats_df = away_stats_df.sort_values(['game_id', 'PosNew'])

In [16]:
home_stats_df.head(5)

,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
19552,EPL,2020-2021,Fulham,2020-09-12,A,5.0,0.067972,0.035714,0.000000,0.0,2.430210,1.010394,0.051843,0.000000,0.114798,0.001613,0.001613,0.001111,0.085177,0.0,1.521941,1.803635,0.074066,1.851767,0.196518,0.137020,0.196518,0.051843,0.016129,0.016129,0.000000,0.035714,0.051843,0.119816,0.067972,1.010394,0.095212,0.233052,0.375832,0.401510,0.114798,0.035714,0.000000,0.079083,0.000000,0.051843,0.027240,0.0,0.0,0.095212,0.164260,0.016129,0.074066,0.016129,0.0,0.0,0.0,0.267127,0.043369,0.051843,1.521941,1.803635,23.119304,4.684485,0.712007,0.774962,0.640655,0.763108,0.069048,0.154224,0.0,0.001111,0.057937,0.082540,0.011111,0.011111,0.074066,1.803635,1.711879,0.091756,0.027240,0.0,0.398054,0.057937,0.074066,0.000000,0.000000,0.000000,0.0,1.430184,0.222683,0.150768,0.309421,1.404019,0.022222,0.000000,0.035714,1.521941,0.000000,0.035714,0.071429,0.027240,2.430210,0.115873,0.329595,1.191039,1.018536,0.032258,2.365694,0.137020,0.0,9.987097,4.417025,0.038351,0.000000,0.138582,0.147056,2.437122,1.880082,0.163185,0,0.0,0,Home
19553,EPL,2020-2021,Fulham,2020-09-12,D,4.0,0.011111,0.011111,0.011111,0.0,3.077778,0.433333,0.077778,0.011111,0.122222,0.000000,0.000000,0.000000,0.011111,0.0,2.344444,2.788889,0.100000,1.766667,0.177778,0.000000,0.000000,0.077778,0.044444,0.033333,0.044444,0.000000,0.044444,0.066667,0.022222,0.433333,0.111111,0.255556,0.144444,0.033333,0.122222,0.033333,0.011111,0.088889,0.011111,0.088889,0.100000,0.0,0.0,0.055556,0.011111,0.011111,0.033333,0.044444,0.0,0.0,0.0,0.300000,0.022222,0.022222,2.344444,2.788889,45.166667,16.144444,0.966667,1.044444,1.033333,1.155556,0.322222,0.477778,0.0,0.000000,0.000000,0.133333,0.000000,0.000000,0.100000,2.788889,2.488889,0.300000,0.066667,0.0,0.244444,0.022222,0.033333,0.011111,0.011111,0.000000,0.0,2.066667,0.377778,0.344444,1.155556,1.311111,0.088889,0.222222,0.000000,2.344444,0.022222,0.066667,0.011111,0.088889,3.077778,0.377778,1.322222,1.455556,0.466667,0.011111,2.800000,0.000000,0.0,8.966667,4.555556,0.066667,0.000000,0.011111,0.000000,2.233333,2.188889,0.022222,0,0.0,0,Home
19554,EPL,2020-2021,Fulham,2020-09-12,G,1.0,0.000000,0.000000,0.000000,0.0,0.311111,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211111,0.288889,0.000000,0.144444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.011111,0.022222,0.000000,0.022222,0.000000,0.000000,0.000000,

In [17]:
away_stats_df.head(5)

,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
18641,EPL,2020-2021,Arsenal,2020-09-12,A,5.0,0.083256,0.047183,0.011111,0.0,2.365504,1.438114,0.024961,0.000000,0.089922,0.017096,0.017096,0.014103,0.201034,0.040000,1.541163,1.830413,0.106667,1.691447,0.213333,0.045478,0.134367,0.024961,0.024961,0.013333,0.011628,0.000000,0.000000,0.033850,0.033850,1.438114,0.421473,0.115401,1.006951,0.315762,0.089922,0.000000,0.0,0.089922,0.000000,0.024961,0.013333,0.000000,0.0,0.047183,0.095401,0.022739,0.113333,0.024961,0.0,0.0,0.0,0.187700,0.011111,0.000000,1.541163,1.830413,24.151163,5.367390,0.806434,0.943023,0.515401,0.559845,0.196072,0.243773,0.026667,0.014103,0.129406,0.064444,0.091111,0.024444,0.106667,1.830413,1.645452,0.184961,0.026667,0.000000,0.277623,0.038295,0.113333,0.146667,0.066667,0.013333,0.0,1.453463,0.187545,0.189406,0.633850,1.138269,0.011111,0.000000,0.036072,1.541163,0.011111,0.023256,0.011628,0.084961,2.365504,0.013333,0.399406,0.912196,1.152196,0.365323,2.167209,0.056589,0.000000,7.332894,3.707907,0.062222,0.035556,0.102739,0.059328,2.791111,1.851292,0.418501,0,3.0,1,Away
18642,EPL,2020-2021,Arsenal,2020-09-12,D,3.0,0.011111,0.011111,0.000000,0.0,3.500000,0.322222,0.055556,0.011111,0.022222,0.001111,0.001111,0.000000,0.022222,0.011111,3.011111,3.288889,0.122222,2.600000,0.133333,0.033333,0.044444,0.055556,0.033333,0.044444,0.000000,0.011111,0.022222,0.033333,0.011111,0.322222,0.088889,0.200000,0.111111,0.011111,0.022222,0.000000,0.0,0.022222,0.011111,0.066667,0.100000,0.011111,0.0,0.066667,0.033333,0.000000,0.022222,0.033333,0.0,0.0,0.0,0.200000,0.022222,0.044444,3.011111,3.288889,64.588889,14.488889,0.722222,0.766667,1.800000,1.877778,0.488889,0.633333,0.000000,0.000000,0.000000,0.122222,0.011111,0.000000,0.122222,3.288889,3.155556,0.133333,0.033333,0.000000,0.322222,0.088889,0.022222,0.000000,0.000000,0.000000,0.0,2.688889,0.288889,0.311111,1.944444,1.166667,0.055556,0.100000,0.011111,3.011111,0.000000,0.066667,0.000000,0.022222,3.500000,0.344444,1.833333,1.777778,0.144444,0.022222,3.366667,0.033333,0.000000,11.888889,6.722222,0.011111,0.011111,0.000000,0.011111,3.022222,2.955556,0.000000,0,3.0,1,Away
18643,EPL,2020-2021,Arsenal,2020-09-12,G,1.0,0.000000,0.000000,0.000000,0.0,0.366667,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.355556,0.355556,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011

In [18]:
home_stats_attack_df = home_stats_df[home_stats_df['PosNew'].isin(['A', 'M'])]
home_stats_attack_df = home_stats_attack_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
home_stats_defense_df = home_stats_df[home_stats_df['PosNew'].isin(['D', 'M'])]
home_stats_defense_df = home_stats_defense_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
home_stats_attack_df.head()

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog
2447,EPL,2020-2021,Fulham,2020-09-12,0,0.0,0,Home,9.0,0.130472,0.035714,0.011111,0.0,5.218573,2.367038,0.150079,0.011111,0.275533,0.001613,0.001613,0.001111,0.085177,0.000000,3.530462,4.082502,0.109801,3.384537,0.381503,0.137020,0.196518,0.150079,0.089740,0.100851,0.013514,0.035714,0.114343,0.351760,0.237417,2.367038,0.468336,0.558690,1.165021,0.643327,0.275533,0.049228,0.0,0.226306,0.011111,0.161190,0.049462,0.000000,0.0,0.144462,0.186482,0.016129,0.096288,0.089740,0.000000,0.0,0.0,0.528126,0.054480,0.051843,3.530462,4.082502,55.740137,10.037338,1.636407,1.739902,1.600152,1.843063,0.146825,0.341349,0.000000,0.001111,0.057937,0.254011,0.011111,0.011111,0.109801,4.082502,3.935190,0.147312,0.071685,0.000000,0.692649,0.069048,0.096288,0.011111,0.000000,0.011111,0.0,3.417121,0.349710,0.315670,1.246058,2.690694,0.044444,0.000000,0.057937,3.530462,0.000000,0.098214,0.096053,0.027240,5.218573,0.272067,0.895136,2.869005,1.665795,0.032258,5.109613,0.137020,0.000000,17.618140,7.752273,0.060573,0.000000,0.228109,0.174083,4.826612,4.032559,0.232254
2415,EPL,2020-2021,Crystal Palace,2020-09-12,1,1.0,1,Home,8.0,0.044444,0.022222,0.026199,0.0,4.400447,1.181893,0.102588,0.073231,0.201499,0.005556,0.005556,0.011438,0.222822,0.024810,1.874869,3.063897,0.175110,2.730586,0.123611,0.277778,0.444444,0.102588,0.080365,0.013699,0.044444,0.044444,0.000000,0.055556,0.055556,1.181893,0.355665,0.380475,0.564108,0.237310,0.201499,0.000000,0.0,0.201499,0.073231,0.175818,0.105855,0.000000,0.0,0.190197,0.150300,0.044444,0.232244,0.080365,0.000000,0.0,0.0,1.259972,0.190578,0.404086,1.874869,3.063897,42.181303,15.033582,0.814979,1.004168,0.537149,0.987068,0.486821,0.797361,0.011111,0.011438,0.164679,0.176499,0.011111,0.000000,0.175110,3.063897,2.534511,0.529386,0.194744,0.000000,0.663210,0.311031,0.232244,0.311031,0.199919,0.000000,0.0,1.453787,0.203896,1.406214,0.791206,1.893115,0.187610,0.012500,0.000000,1.874869,0.011111,0.155556,0.022222,0.266967,4.400447,0.128078,0.812258,2.197311,1.630775,0.177778,3.984759,0.277778,0.000000,9.800062,5.043167,0.044444,0.033333,0.181944,0.351199,2.890935,2.088728,0.327778
2498,EPL,2020-2021,Liverpool,2020-09-12,2,4.0,1,Home,8.0,0.210706,0.037607,0.011111,0.0,5.360911,2.243772,0.167505,0.052632,0.186028,0.035088,0.018421,0.012222,0.367613,0.041414,3.159498,3.990558,0.524579,3.447795,0.522188,0.234683,0.330004,0.167505,0.088484,0.070069,0.059829,0.037607,0.026496,0.09743

In [19]:
away_stats_attack_df = away_stats_df[away_stats_df['PosNew'].isin(['A', 'M'])]
away_stats_attack_df = away_stats_attack_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
away_stats_defense_df = away_stats_df[away_stats_df['PosNew'].isin(['D', 'M'])]
away_stats_defense_df = away_stats_defense_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
away_stats_attack_df.head()

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog
2334,EPL,2020-2021,Arsenal,2020-09-12,0,3.0,1,Away,10.0,0.145649,0.047183,0.022222,0.0,6.939435,2.029994,0.157867,0.044444,0.167700,0.021882,0.021882,0.017608,0.295051,0.040000,5.098428,6.101781,0.384017,4.632045,0.531709,0.162145,0.295478,0.157867,0.145047,0.026154,0.106072,0.025641,0.025641,0.092824,0.067183,2.029994,0.639849,0.294888,1.304814,0.430292,0.167700,0.011111,0.0,0.156589,0.044444,0.202312,0.024444,0.0,0.0,0.080517,0.132153,0.022739,0.192821,0.145047,0.000000,0.000000,0.0,0.711632,0.022222,0.022222,5.098428,6.101781,92.496035,21.042604,2.086349,2.396869,2.156426,2.447024,0.665730,0.816850,0.026667,0.017608,0.153337,0.467009,0.159487,0.035556,0.384017,6.101781,5.665110,0.436671,0.145043,0.023932,0.925913,0.188295,0.192821,0.146667,0.066667,0.013333,0.000000,4.390215,0.939682,0.771884,1.639833,4.092970,0.070085,0.133333,0.143337,5.098428,0.035043,0.056589,0.022739,0.392226,6.939435,0.096239,1.377183,3.538265,2.294077,0.589255,6.487722,0.173256,0.083333,23.043150,14.736112,0.130598,0.118889,0.136072,0.059328,6.496667,5.357702,0.496279
2580,EPL,2020-2021,Southampton,2020-09-12,1,0.0,0,Away,8.0,0.089683,0.045238,0.000000,0.0,4.828947,1.596032,0.338221,0.011111,0.162364,0.007937,0.007937,0.003413,0.114745,0.000000,2.349749,3.343442,0.190476,2.571429,0.240267,0.104762,0.294444,0.338221,0.113952,0.068713,0.068713,0.200794,0.200000,0.272807,0.072807,1.596032,0.519549,0.347076,0.387135,0.861821,0.162364,0.000000,0.0,0.162364,0.011111,0.349332,0.011111,0.0,0.0,0.082665,0.198079,0.011111,0.093776,0.113952,0.000000,0.000000,0.0,0.843442,0.372222,0.477444,2.349749,3.343442,38.279950,9.114077,1.313325,1.871721,0.724520,0.939724,0.233333,0.384712,0.000000,0.003413,0.034127,0.156349,0.056349,0.011111,0.190476,3.343442,3.243442,0.100000,0.033333,0.000000,0.681078,0.079825,0.093776,0.033333,0.011111,0.022222,0.000000,2.007101,0.530869,0.805472,0.542774,2.214160,0.192063,0.011111,0.022222,2.349749,0.000000,0.022222,0.055556,0.162364,4.828947,0.022222,0.560109,2.564119,1.913116,0.337302,4.706725,0.104762,0.000000,9.621930,4.290769,0.022222,0.011905,0.389683,0.249332,4.029156,3.318254,0.389808
2465,EPL,2020-2021,Leeds United,2020-09-12,2,3.0,0,Away,9.0,0.067509,0.040005,0.000000,0.0,4.364052,2.372125,0.323941,0.023611,0.221470,0.005640,0.005640,0.003333,0.068056,0.045833,2.068930,2.971272,0.117236,1.967029,0.281762,0.168056,0.208060,0.323941,0.123338,0.260325,0.034722,0.028893,0.08

In [20]:
away_stats_defense_df[away_stats_defense_df.isna().any(axis=1)]

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog


In [21]:
home_stats_attack_df_78 = home_stats_attack_df[home_stats_attack_df['Season'] == '2017-2018']
home_stats_defense_df_78 = home_stats_defense_df[home_stats_defense_df['Season'] == '2017-2018']
away_stats_attack_df_78 = away_stats_attack_df[away_stats_attack_df['Season'] == '2017-2018']
away_stats_defense_df_78 = away_stats_defense_df[away_stats_defense_df['Season'] == '2017-2018']

In [22]:
len(home_stats_attack_df_78), len(home_stats_defense_df_78), len(away_stats_attack_df_78), len(away_stats_defense_df_78)

(1826, 1826, 1826, 1826)

In [23]:
defensive_columns = [el for el in list(home_stats_attack_df_78.columns) if el.startswith('Tackles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Vs_Dribbles')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pressures_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Blocks_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Int')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Tkl+Int')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Clr')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Err')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Aerial_')] +\
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Short_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Medium_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Long_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_Types')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('SCA_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Carries_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Dribbles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Total_')] 

defensive_columns = list(set(defensive_columns))

In [24]:
offensive_columns = [el for el in list(home_stats_attack_df_78.columns) if el.startswith('SCA_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Carries_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Dribbles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Total_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Short_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Medium_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Long_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_Types')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Outcomes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Touches_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Receiving_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Aerial_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Performance_')]

# offensive_columns.remove('SCA_GCA')

In [25]:
offensive_columns = list(set(offensive_columns + defensive_columns))
defensive_columns = offensive_columns

In [26]:
len(offensive_columns), len(defensive_columns)

(92, 92)

In [27]:
X_train_home_attack_df_78 = home_stats_attack_df_78[offensive_columns]
X_train_home_defense_df_78 = home_stats_defense_df_78[defensive_columns]
y_train_home_df_78 = home_stats_defense_df_78['Goal']

X_train_away_attack_df_78 = away_stats_attack_df_78[offensive_columns]
X_train_away_defense_df_78 = away_stats_defense_df_78[defensive_columns]
y_train_away_df_78 = away_stats_defense_df_78['Goal']

In [28]:
# X_train_home_attack_df_78_means = X_train_home_attack_df_78.mean()
# X_train_home_defense_df_78_means = X_train_home_defense_df_78.mean()
# X_train_away_attack_df_78_means = X_train_away_attack_df_78.mean()
# X_train_away_defense_df_78_means = X_train_away_defense_df_78.mean()

# X_train_home_attack_df_78_stds = X_train_home_attack_df_78.std()
# X_train_home_defense_df_78_stds = X_train_home_defense_df_78.std()
# X_train_away_attack_df_78_stds = X_train_away_attack_df_78.std()
# X_train_away_defense_df_78_stds = X_train_away_defense_df_78.std()

In [29]:
# X_train_home_attack_df_78 = (X_train_home_attack_df_78 - X_train_home_attack_df_78_means) / X_train_home_attack_df_78_stds
# X_train_home_defense_df_78 = (X_train_home_defense_df_78 - X_train_home_defense_df_78_means) / X_train_home_defense_df_78_stds

# X_train_away_attack_df_78 = (X_train_away_attack_df_78 - X_train_away_attack_df_78_means) / X_train_away_attack_df_78_stds
# X_train_away_defense_df_78 = (X_train_away_defense_df_78 - X_train_away_defense_df_78_means) / X_train_away_defense_df_78_stds


In [30]:
y_train_home_df_78.mean()

1.5301204819277108

In [31]:
y_train_away_df_78.mean()

1.17907995618839

In [32]:
assert not np.any(np.isnan(y_train_away_df_78.values))

In [33]:
import torch
import torch.nn as nn

In [34]:
X_h_a = torch.from_numpy(X_train_home_attack_df_78.values).float()
X_h_d = torch.from_numpy(X_train_home_defense_df_78.values).float()
X_a_a = torch.from_numpy(X_train_away_attack_df_78.values).float()
X_a_d = torch.from_numpy(X_train_away_defense_df_78.values).float()

y_h = torch.from_numpy(y_train_home_df_78.values).float().unsqueeze(1)
y_a = torch.from_numpy(y_train_away_df_78.values).float().unsqueeze(1)
y = torch.cat((y_h, y_a), 1)

In [35]:
# MSE loss
def custom_mse(t1, t2):
    y1 = t2[:, 0]
    y2 = t2[:, 1]
    y1_hat = t1[:, 0]
    y2_hat = t1[:, 1]
    diff1 = y1 - y1_hat
    diff2 = y2 - y2_hat
    return (torch.sum(diff1 * diff1) / diff1.numel()) + (torch.sum(diff2 * diff2) / diff2.numel())

In [36]:
torch.__version__

'1.7.1'

In [37]:
# %%writefile best_model.py

class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        
        self.off_rating = nn.Sequential(
            nn.Linear(92,5),
            nn.BatchNorm1d(5),
            nn.ReLU(),
            nn.Linear(5, 1),
            nn.ReLU()
        )
        self.def_rating = nn.Sequential(
            nn.Linear(92,5),
            nn.BatchNorm1d(5),
            nn.ReLU(),
            nn.Linear(5, 1),
            nn.ReLU()
        )
    
    def forward(self, x_h_a, x_h_d, x_a_a, x_a_d):
        r_h_a = self.off_rating(x_h_a)
#         print(r_h_a.shape)
        r_a_d = self.def_rating(x_a_d)
#         print(r_a_d.shape)
        r_a_a = self.off_rating(x_a_a)
#         print(r_a_a.shape)
        r_h_d = self.def_rating(x_h_d)
#         print(r_h_d.shape)
        
        p1 = 1.5438116100766703 * r_h_a * r_a_d
#         print(p1.shape)
        p2 = 1.17907995618839 * r_a_a * r_h_d
#         print(p2.shape)
#         print(torch.cat((p1, p2), 1).shape)
        
        return torch.cat((p1, p2), 1)

In [41]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

dataset = TensorDataset(X_h_a, X_h_d, X_a_a, X_a_d, y)
train_set, val_set = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset) - int(0.8*len(dataset))])
train_loader = DataLoader(
    train_set,
    batch_size=64
)
val_loader = DataLoader(
    val_set,
    batch_size=64
)

In [44]:
# Repeat for given number of epochs
m = model()
opt = torch.optim.SGD(m.parameters(), lr=1e-5)

best_val_loss = float('inf')
best_m = None

for epoch in range(1000):
    m = m.train()
    train_losses = []

    # Train with batches of data
    for x_h_a, x_h_d, x_a_a, x_a_d, y_e in train_loader:

        # 1. Generate predictions
        pred = m(x_h_a, x_h_d, x_a_a, x_a_d)

        # 2. Calculate loss
        loss = custom_mse(pred, y_e)
        
        train_losses.append(loss.item())

        # 3. Compute gradients
        loss.backward()

        # 4. Update parameters using gradients
        opt.step()

        # 5. Reset the gradients to zero
        opt.zero_grad()

    m = m.eval()
    val_losses = []
    with torch.no_grad():
        for x_h_a, x_h_d, x_a_a, x_a_d, y_e in val_loader:

            # 1. Generate predictions
            pred = m(x_h_a, x_h_d, x_a_a, x_a_d)

            # 2. Calculate loss
            loss = custom_mse(pred, y_e)
            
            val_losses.append(loss.item())
    
    if np.mean(val_losses) < best_val_loss:
        best_val_loss = np.mean(val_losses)
        best_m = copy.deepcopy(m)
        

    # Print the progress
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f} Val Loss: {:.4f}'.format(epoch+1, 1000, np.mean(train_losses), np.mean(val_losses)))

Epoch [100/1000], Loss: 6.4309 Val Loss: 6.7371
Epoch [200/1000], Loss: 6.0614 Val Loss: 6.3338
Epoch [300/1000], Loss: 5.6086 Val Loss: 5.8333
Epoch [400/1000], Loss: 5.1132 Val Loss: 5.2974
Epoch [500/1000], Loss: 4.6527 Val Loss: 4.8259
Epoch [600/1000], Loss: 4.2462 Val Loss: 4.4401
Epoch [700/1000], Loss: 3.8812 Val Loss: 4.0654
Epoch [800/1000], Loss: 3.4985 Val Loss: 3.7045
Epoch [900/1000], Loss: 3.2624 Val Loss: 3.4899
Epoch [1000/1000], Loss: 3.1000 Val Loss: 3.3426


In [45]:
# best_m.train()
torch.save(best_m.state_dict(), 'best_model6.pth')
m = copy.deepcopy(best_m)
m = m.eval()

In [41]:
m = model()
m.load_state_dict(torch.load('best_model6.pth'))

m = m.eval()

In [46]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [47]:
custom_mse(m(X_h_a, X_h_d, X_a_a, X_a_d).detach(), y)

tensor(3.1231)

In [48]:
home_stats_attack_df_79 = home_stats_attack_df
home_stats_defense_df_79 = home_stats_defense_df
away_stats_attack_df_79 = away_stats_attack_df
away_stats_defense_df_79 = away_stats_defense_df

In [49]:
ratings = pd.concat([home_stats_attack_df_79, away_stats_attack_df_79], ignore_index = True)
ratings['o_rating'] =  m.off_rating(torch.from_numpy(ratings[offensive_columns].values).float()).detach().squeeze(1).numpy()
ratings_o = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'isHome']].copy()

ratings = pd.concat([home_stats_defense_df_79, away_stats_defense_df_79], ignore_index = True)
ratings['d_rating'] =  m.def_rating(torch.from_numpy(ratings[defensive_columns].values).float()).detach().squeeze(1).numpy()
ratings_d = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'd_rating', 'isHome']].copy()

len(ratings_o), len(ratings_d)


(13958, 13958)

In [50]:
ratings = pd.merge(ratings_o, ratings_d, how='inner', on=['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome'])

In [51]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['Season', 'League', 'Team']).mean()
ratings_new['o_rating'].groupby('League', group_keys=False).nlargest(5)

Season     League      Team           
2017-2018  Bundesliga  Bayern Munich      1.420791
                       RB Leipzig         1.145466
                       M'Gladbach         1.108071
                       Schalke 04         1.086729
                       Dortmund           1.079690
           EPL         Manchester City    1.608587
                       Tottenham          1.321365
                       Chelsea            1.293880
                       Manchester Utd     1.274095
                       Arsenal            1.233255
           LaLiga      Real Madrid        1.556341
                       Barcelona          1.387245
                       Las Palmas         1.098385
                       Villarreal         1.089810
                       Sevilla            1.081337
           Ligue1      Paris S-G          1.691493
                       Nice               1.309449
                       Lyon               1.218545
                       Marseille          1

In [52]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['League', 'Season', 'Team']).mean()
ratings_new['d_rating'].groupby('League', group_keys=False).nlargest(5)

League      Season     Team           
Bundesliga  2017-2018  Bayern Munich      0.886143
                       M'Gladbach         0.848775
                       Schalke 04         0.846041
                       Mainz 05           0.839525
                       Augsburg           0.819822
EPL         2017-2018  Manchester City    0.977843
                       Liverpool          0.926891
                       Tottenham          0.898677
                       Bournemouth        0.894380
                       Newcastle Utd      0.886866
LaLiga      2017-2018  Real Sociedad      0.927231
                       Levante            0.906574
                       Real Madrid        0.901340
                       Valencia           0.861574
                       Alavés             0.853123
Ligue1      2017-2018  Paris S-G          1.049898
                       Lille              1.005186
                       Montpellier        0.937051
                       Lyon               0

In [53]:
# ratings = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'isHome']]
# ratings['o_rating'] = - ratings['o_rating']
# ratings['d_rating'] = - ratings['d_rating']
ratings = ratings.sort_values('Date')
ratings

,League,Season,Team,Date,game_id,Goal,Win,o_rating,isHome,d_rating
2467,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,1.211652,Home,1.166637
9446,Ligue1,2017-2018,Toulouse,2017-08-04,2662,2.0,0,0.975881,Away,1.103493
9448,Ligue1,2017-2018,Nice,2017-08-05,2664,0.0,0,1.964235,Away,0.882480
9449,Ligue1,2017-2018,Caen,2017-08-05,2665,0.0,0,0.874958,Away,0.723319
9450,Ligue1,2017-2018,Guingamp,2017-08-05,2666,3.0,1,0.905422,Away,0.949315
...,...,...,...,...,...,...,...,...,...,...
12817,LaLiga,2020-2021,Valladolid,2021-04-28,6122,2.0,0,1.583753,Away,1.328635
12818,LaLiga,2020-2021,Granada,2021-04-29,6123,2.0,1,0.832787,Away,1.011826
5839,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.444777,Home,1.174736
328,EPL,2020-2021,Southampton,2021-04-30,330,1.0,0,0.795919,Home,0.751859


In [54]:
ratings['AvgORating'] = ratings.groupby(['Team'])['o_rating'].transform(lambda x: x.rolling(30, 1).mean().shift(1, fill_value=0))
ratings['AvgDRating'] = ratings.groupby(['Team'])['d_rating'].transform(lambda x: x.rolling(30, 1).mean().shift(1, fill_value=0))


In [55]:
ratings

,League,Season,Team,Date,game_id,Goal,Win,o_rating,isHome,d_rating,AvgORating,AvgDRating
2467,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,1.211652,Home,1.166637,0.000000,0.000000
9446,Ligue1,2017-2018,Toulouse,2017-08-04,2662,2.0,0,0.975881,Away,1.103493,0.000000,0.000000
9448,Ligue1,2017-2018,Nice,2017-08-05,2664,0.0,0,1.964235,Away,0.882480,0.000000,0.000000
9449,Ligue1,2017-2018,Caen,2017-08-05,2665,0.0,0,0.874958,Away,0.723319,0.000000,0.000000
9450,Ligue1,2017-2018,Guingamp,2017-08-05,2666,3.0,1,0.905422,Away,0.949315,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12817,LaLiga,2020-2021,Valladolid,2021-04-28,6122,2.0,0,1.583753,Away,1.328635,0.975218,0.766955
12818,LaLiga,2020-2021,Granada,2021-04-29,6123,2.0,1,0.832787,Away,1.011826,0.797808,0.759997
5839,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.444777,Home,1.174736,1.746556,0.987971
328,EPL,2020-2021,Southampton,2021-04-30,330,1.0,0,0.795919,Home,0.751859,0.947131,0.755476


In [56]:
ratings_home = ratings[ratings['isHome'] == 'Home'][['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'AvgORating', 'AvgDRating']]
ratings_away = ratings[ratings['isHome'] == 'Away'][['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'AvgORating', 'AvgDRating']]


df89 = pd.merge(ratings_home, ratings_away, on = ['League', 'Season', 'Date', 'game_id'], how='inner', suffixes=('_Home', '_Away'),)
final_data = df89.copy()

df89 = df89[df89['Season'] == '2018-2019']


In [57]:
final_data

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,AvgORating_Home,AvgDRating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away,AvgORating_Away,AvgDRating_Away
0,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,1.211652,1.166637,0.000000,0.000000,Toulouse,2.0,0,0.975881,1.103493,0.000000,0.000000
1,Ligue1,2017-2018,Paris S-G,2017-08-05,2663,2.0,1,1.795054,0.950961,0.000000,0.000000,Amiens,0.0,0,0.727917,1.035919,0.000000,0.000000
2,Ligue1,2017-2018,Saint-Étienne,2017-08-05,2664,1.0,1,0.770151,0.973147,0.000000,0.000000,Nice,0.0,0,1.964235,0.882480,0.000000,0.000000
3,Ligue1,2017-2018,Montpellier,2017-08-05,2665,1.0,1,0.614491,0.658050,0.000000,0.000000,Caen,0.0,0,0.874958,0.723319,0.000000,0.000000
4,Ligue1,2017-2018,Metz,2017-08-05,2666,1.0,0,1.092208,0.643587,0.000000,0.000000,Guingamp,3.0,1,0.905422,0.949315,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,LaLiga,2020-2021,Eibar,2021-04-26,6121,0.0,0,0.658192,0.630045,0.833002,0.769441,Real Sociedad,1.0,1,0.636074,0.695159,1.000492,0.759078
6975,EPL,2020-2021,Leicester City,2021-04-26,329,2.0,1,1.320532,0.703995,1.094266,0.755262,Crystal Palace,1.0,0,0.891666,0.809332,0.903511,0.797465
6976,LaLiga,2020-2021,Athletic Club,2021-04-28,6122,2.0,0,1.097462,0.754600,1.030560,0.774980,Valladolid,2.0,0,1.583753,1.328635,0.975218,0.766955
6977,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.444777,1.174736,1.746556,0.987971,Granada,2.0,1,0.832787,1.011826,0.797808,0.759997


In [58]:
final_data['Season'].value_counts()

2017-2018    1826
2018-2019    1826
2019-2020    1723
2020-2021    1604
Name: Season, dtype: int64

In [59]:
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [60]:
# Confirming with the results shown in class
cols_for_pred = [
    'I(1.5* (AvgORating_Home * AvgDRating_Away) - 1.1*(AvgORating_Away * AvgDRating_Home))', 
#     'I(AvgORating_Home - AvgORating_Away)',
#     'I(AvgDRating_Home - AvgDRating_Away)',
#     'I(o_rating_Home - o_rating_Away)',
#     'I(d_rating_Home - d_rating_Away)'
]

glm_str = "Win_Home ~ " + '+'.join(cols_for_pred)

result_glm = smf.glm(glm_str, 
             data = df89, 
             family = sm.families.Binomial()
).fit()
result_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Win_Home   No. Observations:                 1826
Model:                            GLM   Df Residuals:                     1824
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1197.9
Date:                Mon, 03 May 2021   Deviance:                       2395.8
Time:                        20:38:45   Pearson chi2:                 1.83e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
============================================================================================================================================================
                                                                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                   -0.7807      0.075    -10.469      0.000      -0.927      -0.635
I(1.5 * (AvgORating_Home * AvgDRating_Away) - 1.1 * (AvgORating_Away * AvgDRating_Home))     1.6602      0.164     10.108      0.000       1.338       1.982
============================================================================================================================================================
"""

In [61]:
print("Test Brier Score: ", brier_score_loss(df89['Win_Home'], result_glm.predict(df89)))


Test Brier Score:  0.23191556765989096


In [62]:
result_glm.predict(df89)

1826    0.548388
1827    0.627932
1828    0.285929
1829    0.556135
1830    0.261644
          ...   
3647    0.655025
3648    0.409548
3649    0.432013
3650    0.436038
3651    0.615128
Length: 1826, dtype: float64

In [63]:
df89['Win_Home']

1826    1
1827    1
1828    0
1829    1
1830    0
       ..
3647    1
3648    0
3649    0
3650    1
3651    1
Name: Win_Home, Length: 1826, dtype: int64

In [64]:
print("Testing Acc: ", accuracy_score(df89['Win_Home'], 1 * (result_glm.predict(df89) > 0.5)))


Testing Acc:  0.627053669222344


In [65]:
# Confirming with the results shown in class
cols_for_pred = [
    'I(AvgORating_Home - AvgORating_Away)', 
    'I(AvgDRating_Home - AvgDRating_Away)'
]

glm_str = "Win_Home ~ " + '+'.join(cols_for_pred)

result_glm = smf.glm(glm_str, 
             data = df89, 
             family = sm.families.Binomial()
).fit()
result_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Win_Home   No. Observations:                 1826
Model:                            GLM   Df Residuals:                     1823
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1183.4
Date:                Mon, 03 May 2021   Deviance:                       2366.8
Time:                        20:39:06   Pearson chi2:                 1.84e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -0.2252      0.049     -4.596      0.000      -0.321      -0.129
I(AvgORating_Home - AvgORating_Away)     1.9385      0.184     10.557      0.000       1.579       2.298
I(AvgDRating_Home - AvgDRating_Away)     0.3754      0.547      0.686      0.493      -0.697       1.448
========================================================================================================
"""

In [66]:
print("Test Brier Score: ", brier_score_loss(df89['Win_Home'], result_glm.predict(df89)))


Test Brier Score:  0.2279911353845814


In [67]:
print("Testing Acc: ", accuracy_score(df89['Win_Home'], 1 * (result_glm.predict(df89) > 0.5)))


Testing Acc:  0.6265060240963856


In [69]:
final_data.drop(['AvgORating_Home', 'AvgORating_Away', 'AvgDRating_Home', 'AvgDRating_Away'], 1).to_csv('offensive_defensive_ratings6.csv', index=False)


In [70]:
final_data

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,AvgORating_Home,AvgDRating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away,AvgORating_Away,AvgDRating_Away
0,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,1.211652,1.166637,0.000000,0.000000,Toulouse,2.0,0,0.975881,1.103493,0.000000,0.000000
1,Ligue1,2017-2018,Paris S-G,2017-08-05,2663,2.0,1,1.795054,0.950961,0.000000,0.000000,Amiens,0.0,0,0.727917,1.035919,0.000000,0.000000
2,Ligue1,2017-2018,Saint-Étienne,2017-08-05,2664,1.0,1,0.770151,0.973147,0.000000,0.000000,Nice,0.0,0,1.964235,0.882480,0.000000,0.000000
3,Ligue1,2017-2018,Montpellier,2017-08-05,2665,1.0,1,0.614491,0.658050,0.000000,0.000000,Caen,0.0,0,0.874958,0.723319,0.000000,0.000000
4,Ligue1,2017-2018,Metz,2017-08-05,2666,1.0,0,1.092208,0.643587,0.000000,0.000000,Guingamp,3.0,1,0.905422,0.949315,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,LaLiga,2020-2021,Eibar,2021-04-26,6121,0.0,0,0.658192,0.630045,0.833002,0.769441,Real Sociedad,1.0,1,0.636074,0.695159,1.000492,0.759078
6975,EPL,2020-2021,Leicester City,2021-04-26,329,2.0,1,1.320532,0.703995,1.094266,0.755262,Crystal Palace,1.0,0,0.891666,0.809332,0.903511,0.797465
6976,LaLiga,2020-2021,Athletic Club,2021-04-28,6122,2.0,0,1.097462,0.754600,1.030560,0.774980,Valladolid,2.0,0,1.583753,1.328635,0.975218,0.766955
6977,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.444777,1.174736,1.746556,0.987971,Granada,2.0,1,0.832787,1.011826,0.797808,0.759997


In [72]:
final_data['League'].unique()

array(['Ligue1', 'EPL', 'Bundesliga', 'LaLiga', 'SerieA'], dtype=object)